**Un algoritmo basado en PE debe considerar los siguientes aspectos:**
- Representación de los individuos, mutación
- Selección de sobreviviente
- Autoadaptación

In [1]:
from math import exp
from math import sqrt
from math import cos
from math import e
from math import pi
import numpy as np
from numpy import std
from random import randint

# **Resolver**
---


$$
min f(\overrightarrow{x}) = -20 \ast exp \left( -0.2 \ast \sqrt{\frac{1}{n} \sum_{i=1}^{n}x^{2}_{i}}\right) - exp \left( \frac{1}{n} \sum_{i=1}^{n} cos(2 \pi x_{i})  \right) + 20 + e
$$

> La función $f$ es conocida como la **función de Ackley**, su dominio es $|x_{i}| \leq 30$ y su mínimo global está en $x_{i} = 0$ y $f(\overrightarrow{x}) = 0.0$


#### **Consideraciones:**
  - Cuidar que las variables de decisión no salgan del rango especificado en el problema. 
  > Si eso ocurre, deberá contar con un mecanismo que vuelva a generar el valor de la variable o que lo ajuste al rango deseado. 
  >> Explique en su reporte qué mecanismo fue utilizado 

# **Función** 
---

In [2]:
def f(x, n): 
  """
  x = list
  n = int, num of decisition variables
  return: float number
  """
  # Para efectuar la primer sumatoria
  sum1 = 0
  for i in range(n): 
    sum1 += x[i]**2
  sumatoria1 = (1/n)*sum1
  # Para efectuar la segunda sumatoria
  sum2 = 0
  for i in range(n): 
    sum2 += cos(2*pi*x[i])
  sumatoria2 = (1/n)*sum2

  return (-20 * exp(-0.2*sqrt(sumatoria1)) - exp(sumatoria2) + 20 + e)

In [3]:
def fitness(x, n):
  """
  x = list
  n = int, num of decision variables
  return: float number
  """
  return -f(x, n)

**Representación del individuo**
---
Un arreglo de valores flotantes de tamaño 4.
Las primeras dos componentes corresponden a las dos variables del problema.
Las siguentes dos componentes corresponden a los tamaños de paso para la mutación de cada una de las variables del problema.



```
x1 = abs(np.random.uniform(-30, 30))
x2 = abs(np.random.uniform(-30, 30))
o1 = np.random.uniform(0,1)
o2 = np.random.uniform(0,1)


X = [x1, x2, o1, o2]
```



**Población inicial**
---

In [4]:
# Generamos un individuo de la siguiente forma: Utilizando una distribución uniforme,
# generamos dos números aleatorios en el intervalo abs([-30, 30]) y los
# guardamos en las primeras dos posiciones de un arreglo de tamaño 4. 
# Posteriormente, generamos otros dos números aleatorios en el intervalo (0, 1) y
# los guardamos en las últimas dos posiciones de nuestro arreglo. Repetimos el
# mismo procedimiento para generar la cantidad de individuos que se deseen.
# En este caso usaremos una población de tamaño μ = 100.

# La descripción anterior es para n = 2
# en general, el arreglo de representación 
# del individuo es de tamaño 2n
# Donde
# Las primer mitad de componentes corresponden a las n variables del problema.
# Las otra mitad de componentes corresponden a los n tamaños de paso para la mutación
# de cada una de las variables del problema.
def getInitialPopulation(mu, n):
  """
  mu = int, poblation size
  n = int, num of decision variables
  return: list with initial population and their fitness by guy
  """
  parents = []
  for i in range(mu):
    #Generamos un individuo
    guy = np.concatenate((abs(np.random.uniform(-30, 30, n)), np.random.uniform(0, 1, n)))
    #print(guy)
    #print(guy[:n]) # las x
    #print(guy[n:]) # las o
    #print(fitness(guy[:n], n))
    #break

    #Calculamos la aptitud del individuo
    guy = [guy, fitness(guy[:n], n)]
    #print(guy)
    #break

    #Agregamos al individuo a la población "parents"
    parents.append(guy)

  # Regresamos la población generada
  # Arreglo de arreglos, donde un indice corresponde a un individuo formado como: 
  # mitad variables del problema, mitad variables de mutación y fitness
  return parents

In [5]:
#print(getInitialPopulation(100,2))
#population = getInitialPopulation(100,2)

# primer individuo generado
#print(len(population), population[0])

**Mutación**
---

In [6]:
def mutation(guy, n, alpha, epsilon):
  """
  guy = list, guy number any
  n = int, num of decision variables
  alpha = float, mutation parameter
  epsilon = float, tolerance number
  return: list, new guy
  """
  #print(guy, type(guy), n, alpha, epsilon)
  #print(guy)
  #tmp = (1+(alpha*np.random.normal(0, 1, n)))
  #print("tmp", tmp)
  #print("guy", guy[n:])
  #print("result", guy[n:]*tmp)
  #print(np.random.normal(0, 1, n))
  #return (None)
  
  # Mutamos los últimos dos componentes de nuestro arreglo -> (tamaños de paso)
  mutation_sigma = guy[n:]*(1+(alpha*np.random.normal(0, 1, n)))

  
  # Verificamos que los nuevos valores no sean menores a épsilon
  # Si se cumple para alguno de los valores, este será igual a epsilon
  mutation_sigma[mutation_sigma < epsilon] = epsilon

  # Mutamos las variables de decisión a partir de las mutaciones
  mutation_x = guy[:n] + (mutation_sigma*np.random.normal(0, 1, n))

  # Revisamos que las variables de decisión estén dentro de los límites
  # De lo contrario las igualamos al limite
  # PROPONER OTRO CAMINO Y EVALUAR
  mutation_x[mutation_x < -30] = randint(-30, 0)
  mutation_x[mutation_x > 30] = randint(0, 31)

  # Creamos el nuevo Individuo y lo devolvemos
  return [np.concatenate((mutation_x, mutation_sigma)), fitness(mutation_x, n)]

Dado que se trata de un algoritmo pseudoestocastico, ya que tiene componentes deterministas y algunos estocasticos. He elegido evaluar que la variable de decisión de tal forma que si se revasa los limites, se le reasigne un número aleatorio dentro de los limites. Esto con la finalidad de añadir más componentes estocasticos al algoritmo y dejar un poco más de lado lo determinista, tal como sigue:

```
mutation_x[mutation_x < -30] = -30
mutation_x[mutation_x > 30] = 30
```



In [7]:
#guys = getInitialPopulation(100,2)
#print(guys[0][0])
#child = mutation(guys[0][0], num_variables, alpha, epsilon)
#print(child)

**Algoritmo Completo**
---

In [8]:
def PE(n, mu, num_gen, alpha, epsilon):
  """
  n = int, num of decision variables
  mu = int, poblation size
  num_gen = int; generations number
  alpha = float, mutation parameter
  epsilon = float, tolerance number
  return: best solution for the function
  """
  
  parents = getInitialPopulation(mu, n)

  for _ in range(num_gen):
    new_gen = parents.copy()
    for parent in parents:
      # Creamos un hijo
      child = mutation(parent[0], n, alpha, epsilon)
      # Agregamos al hijo a la nueva generación
      new_gen.append(child)

    # En este caso utilizaremos una selección más, por lo tanto
    # ordenamos del peor individuo al mejor individuo 
    # (del menor valor de aptitud al mayor)
    #print(new_gen)
    new_gen = sorted(new_gen, key=lambda individual: individual[-1])
    #print("\n", new_gen)
    
    # Nos quedamos con los mu mejores individuos
    new_gen = new_gen[mu:]
    #print("\n", new_gen)
    #return (None)
    parents = new_gen.copy()
  
  
  #for tmp in parents:
    #print(tmp)
  
  # Devolvemos el mejor
  # parents[-1][0] = lista de los 2n valores (mitad x y mitad o)
  # -parents[-1][1] = resultado del fitness aplicado a las x
  return (parents[-1][0], -parents[-1][1])

In [9]:
#print(PE(num_variables, poblation_size, generations_number, alpha, epsilon))
#sol, fx = PE(num_variables, poblation_size, generations_number, alpha, epsilon)
#Xs = sol[:num_variables]
#Sigmas = sol[num_variables:]
#print(f"Xs: {Xs}\nSigmas: {Sigmas}\nf(x): {round(fx,3)}")

> Su mínimo global está en $x_{i} = 0$ y $f(\overrightarrow{x}) = 0.0$

# **INPUT** 
---
1. La primer línea tendrá el número de variables que tendrá a la función de Ackley $\textbf{n}$

2. La segunda línea tendrá el tamaño de la población y el número máximo de generaciones, *separados por un espacio*

3. La tercer línea tendrá el valor de los parámetros $\alpha$ y $\epsilon$, separados por un espacio

In [10]:
def get_data_by_txt(name_file: str):
  """function to read the input from file txt
  name_file = str with the name file.txt
  return: int num_variables, int poblation_size, int generations_number, float alpha. float epsilon 
  """

  with open('input.txt', 'r') as fichero:
    num_variables = fichero.readline()
    poblation_size, generations_number = fichero.readline().split(' ')
    alpha, epsilon = fichero.readline().split(' ')
    # casteamos al tipo de dato correcto 
    num_variables = int(num_variables)
    poblation_size = int(poblation_size)
    generations_number = int(generations_number)
    alpha = float(alpha)
    epsilon = float(epsilon)

    return (num_variables, poblation_size, generations_number, alpha, epsilon)

In [11]:
#num_variables, poblation_size, generations_number, alpha, epsilon = get_data_by_txt("input.txt")
#print(num_variables, poblation_size, generations_number, alpha, epsilon)

# Main
---

In [18]:
 if __name__ == "__main__":
 
  #file_name = str(input("Nombre del archivo: "))
  #num_variables, poblation_size, generations_number, alpha, epsilon = get_data_by_txt(file_name)
  opt = int(input("1) Datos a mano\n2) Datos por txt\nIntroduce un número: "))
  if opt == 1:
    num_variables = int(input("Número de variables: "))
    poblation_size = int(input("Tamaño de la población: "))
    generations_number = int(input("Número de generaciones: "))
    alpha = float(input("Valor de alpha: "))
    epsilon = float(input("Valor de epsilon: "))
  
  elif opt == 2: 
    num_variables, poblation_size, generations_number, alpha, epsilon = get_data_by_txt("input.txt")
  else: 
    print("Opción no disponible")
  # Nota: Se deben reportar los resultados del algoritmo para las
  # siguientes instancias: 5, 10 y 20 variables de decisión.
  # Recuerde que puede variar los parámetros del algoritmo (tamaño de la 
  # población, número de generaciones").

  M = int(input("Número de ejecuciones: "))
    
  # ------------------------------------------PARTE 2
  #1. Mejor solución encontrada considerando las M ejecuciones.
  #2. Peor solución encontrada considerando las M ejecuciones.
  #3. Solución que corresponde a la mediana considerando las M ejecuciones.
  #4. Media del valor de la función objetivo considerando las M ejecuciones.
  #5. Desviación est ́andar del valor de la función objetivo considerando las M ejecuciones.

  solutions = []
  conteo = 0
  #En los primeros tres puntos indica tanto el valor de x (lista de objetos y su
  #correspondiente cadena binaria) como el valor de las funciones f (valor de la mochila) y g (peso de la mochila).
  while conteo != M:
    sol, fx = PE(num_variables, poblation_size, generations_number, alpha, epsilon)
    #Xs = sol[:num_variables]
    #Sigmas = sol[num_variables:]
    solutions.append([sol, fx])
    conteo += 1  

     
  
  my_f_sol = [x[-1] for x in solutions] # fitness/ la evaluación de las x en la funcióón objetivo
  
    
  # Ordenamos del peor individuo al mejor individuo con base en el fitness 
  solutions = sorted(solutions, key=lambda value: value[-1]) 

  # mejor solución
  best_Xs = solutions[0][0][:num_variables]
  best_Sigmas = solutions[0][0][num_variables:]
  best_fx = solutions[0][-1]
  print(f"\nMEJOR SOLUCION\n X's: {best_Xs}\nSigmas: {best_Sigmas}\nf(x): {round(best_fx,3)}")
  # peor solución
  worst_Xs = solutions[-1][0][:num_variables]
  worst_Sigmas = solutions[-1][0][num_variables:]
  worst_fx = solutions[-1][-1]
  print(f"PEOR SOLUCION\n X's: {worst_Xs}\nSigmas: {worst_Sigmas}\nf(x): {round(worst_fx,3)}")  
  print(f"MEDIANA\n X's: {solutions[M//2][0][:num_variables]}\nSigmas: {solutions[M//2][0][num_variables:]}\nf(x): {round(solutions[M//2][-1],3)}")
  print("MEDIA DE LAS FITNESS\n", np.mean(my_f_sol))
  print("DESVIACION ESTANDAR DE LAS FITNESS\n", np.std(my_f_sol))



1) Datos a mano
2) Datos por txt
Introduce un número: 1
Número de variables: 10
Número de ejecuciones: 1000

MEJOR SOLUCION
 X's: [ 0.0045152   0.00075988  0.00475325 -0.00038708 -0.02435038  0.00024418
 -0.0183235   0.02604997  0.01117783 -0.02369965]
Sigmas: [0.0116969  0.0001     0.0001     0.0001     0.00415816 0.0001
 0.01087061 0.0001     0.0001     0.0001    ]
f(x): 0.074
PEOR SOLUCION
 X's: [ 6.99598738  6.99465073 -1.99849188  1.99743612  3.99772355  9.99308595
 16.98918002  0.99936624 17.98892801  0.99975293]
Sigmas: [0.0001     0.0001     0.0001     0.00022641 0.00032319 0.0001
 0.0001     0.00023391 0.00012132 0.0001    ]
f(x): 16.789
MEDIANA
 X's: [ 9.68096361e-01  9.95061737e-01 -7.96135980e-03  2.14445692e-03
  9.89147263e-01  9.99871351e-01  9.87932166e-01  9.90790922e-01
  2.98374097e+00  4.93476350e+00]
Sigmas: [0.00020917 0.00182665 0.0001     0.00324803 0.00577215 0.00010158
 0.0001     0.01416165 0.00024469 0.0001    ]
f(x): 6.566
MEDIA DE LAS FITNESS
 7.1244126645

# **Resultados**

## n = 5
> Número de ejecuciones = 100
```
MEJOR SOLUCION
 X's: [-1.62036010e-05 -1.71289378e-05  1.41171710e-05 -7.67381288e-06
  1.49993736e-05]
Sigmas: [0.00013667 0.00032539 0.0001     0.0001     0.00017973]
f(x): 0.0
PEOR SOLUCION
 X's: [ 1.09741526e+01  4.98827068e+00  9.97641185e-01 -2.09526719e-05
 -4.05536850e-05]
Sigmas: [0.00011253 0.00011383 0.0001     0.00025668 0.00030095]
f(x): 13.23
MEDIANA
 X's: [ 1.79453613e-05  9.45352089e-01  2.88086745e-05 -1.11987066e-05
  9.45412382e-01]
Sigmas: [0.0001     0.0001     0.00014842 0.00022322 0.00010381]
f(x): 2.317
MEDIA DE LAS FITNESS
 2.893811578715165
DESVIACION ESTANDAR DE LAS FITNESS
 3.112272436973118
```



## n = 10 
> Número de ejecuciones = 100

```
MEJOR SOLUCION
 X's: [ 0.00307628  0.07154816 -0.02416617  0.12001407  0.08271466 -0.01984347
  0.04526667  0.06972563 -0.01238512 -0.02548058]
Sigmas: [1.00000000e-04 1.22149373e-01 2.83086969e-04 2.11376405e-04
 2.44904838e-04 4.61229174e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04]
f(x): 0.412
PEOR SOLUCION
 X's: [12.9891006  -0.9992072   1.99834784  4.99599988  4.99599019  0.99896934
  4.99582709 11.99002795  4.99593678 16.98593202]
Sigmas: [0.00012535 0.0001     0.00010493 0.0001     0.0001     0.00032409
 0.0001     0.0001     0.00027115 0.0001    ]
f(x): 16.281
MEDIANA
 X's: [ 1.0133202   5.96695473  0.08086425  1.01140741 -0.97054949  1.01580088
 -1.04552821  0.03537899 -2.10692036 -1.00289257]
Sigmas: [8.98058808e+00 1.97075710e-04 1.00000000e-04 1.00000000e-04
 1.46333192e-03 1.00000000e-04 2.65729406e-02 3.19947182e-04
 1.00000000e-04 3.46447012e-04]
f(x): 7.109
MEDIA DE LAS FITNESS
 7.445055038682716
DESVIACION ESTANDAR DE LAS FITNESS
 3.9068061374222705
```


 
## n = 20
> Número de ejecuciones = 100

```
MEJOR SOLUCION
 X's: [ 0.17848684  0.86204387 -1.01863474 -1.0724038   0.95890567  4.13661845
  0.39789309  3.0171237   2.72138765 -0.85319077 -0.26925651 -0.99002892
  0.22514868  0.64495898  0.08788658 -1.07513346 -1.11902064  0.95780707
  0.02234067  1.02400942]
Sigmas: [2.77787653e-04 1.00000000e-04 1.20000224e-04 1.00000000e-04
 3.53358654e-04 3.29324077e-04 1.00000000e-04 1.47694548e-04
 4.48607012e-03 1.54648387e-04 1.00000000e-04 1.64589110e-03
 5.02166772e-04 1.00000000e-04 1.03327459e-04 5.26573396e-04
 1.75940957e-01 1.61883317e-04 2.16827288e-03 2.68533671e-04]
f(x): 6.129
PEOR SOLUCION
 X's: [ 1.69844810e+01  1.00035978e+00 -6.00837693e+00  1.79865271e+01
  1.39996076e+01  9.97329973e-01  1.00378733e+00 -9.99670362e+00
  7.99169521e+00  3.00777612e+00  1.99339428e+00  9.99211566e+00
  9.95420990e-01  6.73072306e-03  1.29834270e+01 -1.99420807e+00
  1.29973719e+01  2.09805932e+01  1.99808499e+00  6.99663355e+00]
Sigmas: [0.00048505 0.00322274 0.0001     0.00025293 0.0001     0.0001
 0.00020839 0.00018116 0.0001     0.00018121 0.0001     0.00345614
 0.00012809 0.0009693  0.0001     0.0001     0.00165429 0.01894341
 0.000166   0.00077355]
f(x): 17.242
MEDIANA
 X's: [ 1.71409802e-03  4.99980618e+00  8.10824783e+00  2.01222752e+00
  1.98286999e+00  1.96941753e+00  4.92882805e+00  1.00125457e+01
  1.04543773e+00  7.95362936e+00 -6.95963067e+00  2.18215178e+00
  9.00130432e-01  0.00000000e+00  2.00649718e+00  2.03631858e+00
  1.98008243e+00 -3.99073005e+00 -1.20020373e+01  3.03579056e+00]
Sigmas: [1.86728262e-03 1.00000000e-04 1.00000000e-04 2.76816043e-04
 1.00000000e-04 2.93810399e-03 2.17685902e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.60121033e-03 1.10265797e-04
 1.00000000e-04 4.77007431e+01 1.00000000e-04 1.00000000e-04
 1.66186109e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04]
f(x): 13.006
MEDIA DE LAS FITNESS
 13.077336391770105
DESVIACION ESTANDAR DE LAS FITNESS
 2.3057006800713062
```



He realizado nuevamente la ejecución para un **número de variables** igual a **10** pero ahora para **1000 ejecuciones**, si bien tardo más de 8 minutos en completar la ejecución, no hubo mucha mejora en los resultados: 


```
MEJOR SOLUCION
 X's: [ 0.0045152   0.00075988  0.00475325 -0.00038708 -0.02435038  0.00024418
 -0.0183235   0.02604997  0.01117783 -0.02369965]
Sigmas: [0.0116969  0.0001     0.0001     0.0001     0.00415816 0.0001
 0.01087061 0.0001     0.0001     0.0001    ]
f(x): 0.074
PEOR SOLUCION
 X's: [ 6.99598738  6.99465073 -1.99849188  1.99743612  3.99772355  9.99308595
 16.98918002  0.99936624 17.98892801  0.99975293]
Sigmas: [0.0001     0.0001     0.0001     0.00022641 0.00032319 0.0001
 0.0001     0.00023391 0.00012132 0.0001    ]
f(x): 16.789
MEDIANA
 X's: [ 9.68096361e-01  9.95061737e-01 -7.96135980e-03  2.14445692e-03
  9.89147263e-01  9.99871351e-01  9.87932166e-01  9.90790922e-01
  2.98374097e+00  4.93476350e+00]
Sigmas: [0.00020917 0.00182665 0.0001     0.00324803 0.00577215 0.00010158
 0.0001     0.01416165 0.00024469 0.0001    ]
f(x): 6.566
MEDIA DE LAS FITNESS
 7.124412664578651
DESVIACION ESTANDAR DE LAS FITNESS
 3.8598447151497894
```

### NOTA
> En las corridas mostradas, lo único que varió fue el número de variables de decisión. 
